In [ ]:
# default_exp core

# Pre-reqs

In [5]:
#hide
from google.colab import drive
drive.mount("/content/drive")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
#hide
!pip install nbdev
from nbdev.showdoc import *

In [ ]:
#hide
!pip install nbdev
import os
# define pathway to the weights
weight_filenames = {
    "lcd": "21-2-20-94-universal-lcd.pt",
    "digits":'21-2-25 1k-digits YOLOv5-weights.pt'
    }

resource_folder = "/content/drive/MyDrive/Coding/Roboflow/try it out"

# detectors = []
# for filename in weight_filenames:
# weights_path = os.path.join(resource_folder, weights_filename)



# module name here

> API details.

In [1]:
import os

# safety for re-executions
if not os.path.exists("yolov5"):
  # clone YOLOv5 and reset to a specific git checkpoint that has been verified working
  !git clone https://github.com/ultralytics/yolov5  # clone repo
  !git reset --hard 68211f72c99915a15855f7b99bf5d93f5631330f

# enter the yolov5 directory
%cd yolov5

# install dependencies as necessary
!pip install -qr requirements.txt  # install dependencies (ignore errors)
import torch

from IPython.display import Image, clear_output  # to display images
# from utils.google_utils import gdrive_download  # to download models/datasets

clear_output()

if torch.cuda.is_available():
  print('Setup complete. Using torch %s %s' % (torch.__version__, torch.cuda.get_device_properties(0)))
else:
  raise Exception("You need to enable GPU in this runtime environment")

# return to parent directory
%cd ..

Setup complete. Using torch 1.8.0+cu101 _CudaDeviceProperties(name='Tesla P100-PCIE-16GB', major=6, minor=0, total_memory=16280MB, multi_processor_count=56)
/content


In [2]:
resource_map = {"images": ".jpg", "labels": ".txt"}

In [3]:
import glob

def collect_files(walk_dir):
  res = {}
  for key, extension in resource_map.items():
    resource_generator = glob.iglob(walk_dir + '/**/*' + extension, recursive=True)
    res[key] = [{"pair_id": os.path.basename(x)[:-1*len(extension)], "path": x} for x in resource_generator]
  return res

def matched(res):
  bn = lambda x: set([z["pair_id"] for z in x])
  matched = (bn(res["labels"]).intersection(bn(res["images"])))
  pairs = []
  for resource in matched:
    tmp = {}
    for k in resource_map:
      tmp[k] = [x for x in res[k] if x["pair_id"] == resource][0]
    pairs.append(tmp)
    
  return pairs

In [4]:
default_split_ratio = {
   "train": .7,
   "valid": .2,
   "test": .1 
}

split_ratio = default_split_ratio

In [5]:
repo = "/content/drive/MyDrive/Coding/Roboflow Export (841)"

In [11]:
import math, random

files = matched(collect_files(repo))
random.shuffle(files)

train = math.ceil(len(files) * split_ratio["train"])
valid = train + math.ceil(len(files) * split_ratio["valid"])

split =  {"train": files[:train],
"valid": files[train: valid],
"test": files[valid:]}

assert sum([len(split[x]) for x in split]) == len(files)

[{x: len(split[x])} for x in split]

[{'train': 589}, {'valid': 169}, {'test': 83}]

In [12]:
from os.path import join

def mkdir(dir):
  if not os.path.exists(dir):
    !mkdir "{dir}"


In [13]:
%%time

for setname, pairs in split.items():
  dir = join("./", setname)
  mkdir(dir)
  for pair in pairs:
    for resource, data in pair.items():
      subdir = join(dir, resource)
      mkdir(subdir)
      target = data["path"]
      dest_filename = data["pair_id"]+resource_map[resource]
      destination = join(subdir, dest_filename)
      if not os.path.exists(destination):
        !cp "{target}" "{destination}"



CPU times: user 1.85 s, sys: 3.34 s, total: 5.2 s
Wall time: 1min 27s


In [14]:
lens = {}
for x in split.keys():
  for r in resource_map:
    files = !ls "{x}/{r}"
    lens[f"{x}-{r}"] = len(files)
lens

{'test-images': 161,
 'test-labels': 161,
 'train-images': 769,
 'train-labels': 769,
 'valid-images': 307,
 'valid-labels': 307}

In [15]:
%%writefile ./data.yaml
train: ../train/images
val: ../valid/images

nc: 10
names: ['1', '2', '3', '4', '5', '6', '7', '8', '9', '0']

Overwriting ./data.yaml


#####architecture

In [16]:
# define number of classes based on YAML
import yaml
with open("data.yaml", 'r') as stream:
    num_classes = str(yaml.safe_load(stream)['nc'])

In [17]:
#this is the model configuration we will use for our tutorial 
%cat /content/yolov5/models/yolov5s.yaml

# parameters
nc: 80  # number of classes
depth_multiple: 0.33  # model depth multiple
width_multiple: 0.50  # layer channel multiple

# anchors
anchors:
  - [10,13, 16,30, 33,23]  # P3/8
  - [30,61, 62,45, 59,119]  # P4/16
  - [116,90, 156,198, 373,326]  # P5/32

# YOLOv5 backbone
backbone:
  # [from, number, module, args]
  [[-1, 1, Focus, [64, 3]],  # 0-P1/2
   [-1, 1, Conv, [128, 3, 2]],  # 1-P2/4
   [-1, 3, C3, [128]],
   [-1, 1, Conv, [256, 3, 2]],  # 3-P3/8
   [-1, 9, C3, [256]],
   [-1, 1, Conv, [512, 3, 2]],  # 5-P4/16
   [-1, 9, C3, [512]],
   [-1, 1, Conv, [1024, 3, 2]],  # 7-P5/32
   [-1, 1, SPP, [1024, [5, 9, 13]]],
   [-1, 3, C3, [1024, False]],  # 9
  ]

# YOLOv5 head
head:
  [[-1, 1, Conv, [512, 1, 1]],
   [-1, 1, nn.Upsample, [None, 2, 'nearest']],
   [[-1, 6], 1, Concat, [1]],  # cat backbone P4
   [-1, 3, C3, [512, False]],  # 13

   [-1, 1, Conv, [256, 1, 1]],
   [-1, 1, nn.Upsample, [None, 2, 'nearest']],
   [[-1, 4], 1, Concat, [1]],  # cat backbone P3
   [-1, 3, C

In [18]:
#customize iPython writefile so we can write variables
from IPython.core.magic import register_line_cell_magic

@register_line_cell_magic
def writetemplate(line, cell):
    with open(line, 'w') as f:
        f.write(cell.format(**globals()))

In [19]:
%%writetemplate /content/yolov5/models/custom_yolov5s.yaml

# parameters
nc: {num_classes}  # number of classes
depth_multiple: 0.33  # model depth multiple
width_multiple: 0.50  # layer channel multiple

# anchors
anchors:
  - [10,13, 16,30, 33,23]  # P3/8
  - [30,61, 62,45, 59,119]  # P4/16
  - [116,90, 156,198, 373,326]  # P5/32

# YOLOv5 backbone
backbone:
  # [from, number, module, args]
  [[-1, 1, Focus, [64, 3]],  # 0-P1/2
   [-1, 1, Conv, [128, 3, 2]],  # 1-P2/4
   [-1, 3, BottleneckCSP, [128]],
   [-1, 1, Conv, [256, 3, 2]],  # 3-P3/8
   [-1, 9, BottleneckCSP, [256]],
   [-1, 1, Conv, [512, 3, 2]],  # 5-P4/16
   [-1, 9, BottleneckCSP, [512]],
   [-1, 1, Conv, [1024, 3, 2]],  # 7-P5/32
   [-1, 1, SPP, [1024, [5, 9, 13]]],
   [-1, 3, BottleneckCSP, [1024, False]],  # 9
  ]

# YOLOv5 head
head:
  [[-1, 1, Conv, [512, 1, 1]],
   [-1, 1, nn.Upsample, [None, 2, 'nearest']],
   [[-1, 6], 1, Concat, [1]],  # cat backbone P4
   [-1, 3, BottleneckCSP, [512, False]],  # 13

   [-1, 1, Conv, [256, 1, 1]],
   [-1, 1, nn.Upsample, [None, 2, 'nearest']],
   [[-1, 4], 1, Concat, [1]],  # cat backbone P3
   [-1, 3, BottleneckCSP, [256, False]],  # 17 (P3/8-small)

   [-1, 1, Conv, [256, 3, 2]],
   [[-1, 14], 1, Concat, [1]],  # cat head P4
   [-1, 3, BottleneckCSP, [512, False]],  # 20 (P4/16-medium)

   [-1, 1, Conv, [512, 3, 2]],
   [[-1, 10], 1, Concat, [1]],  # cat head P5
   [-1, 3, BottleneckCSP, [1024, False]],  # 23 (P5/32-large)

   [[17, 20, 23], 1, Detect, [nc, anchors]],  # Detect(P3, P4, P5)
  ]

In [20]:
# train yolov5s on custom data for 100 epochs
# time its performance
%%time
%cd /content/yolov5/
!python train.py --img 416 --batch 16 --epochs 10 --data '../data.yaml' --cfg ./models/custom_yolov5s.yaml --weights '' --name yolov5s_results  --cache

/content/yolov5
remote: Enumerating objects: 11, done.
remote: Counting objects: 100% (11/11), done.
remote: Compressing objects: 100% (11/11), done.
remote: Total 11 (delta 3), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (11/11), done.
From https://github.com/ultralytics/yolov5
   747c265..6f718ce  master     -> origin/master
github: ⚠️ WARNING: code is out of date by 4 commits. Use 'git pull' to update or 'git clone https://github.com/ultralytics/yolov5' to download latest.
YOLOv5 v4.0-130-g747c265 torch 1.8.0+cu101 CUDA:0 (Tesla P100-PCIE-16GB, 16280.875MB)

Namespace(adam=False, batch_size=16, bucket='', cache_images=True, cfg='./models/custom_yolov5s.yaml', data='../data.yaml', device='', entity=None, epochs=10, evolve=False, exist_ok=False, global_rank=-1, hyp='data/hyp.scratch.yaml', image_weights=False, img_size=[416, 416], linear_lr=False, local_rank=-1, log_artifacts=False, log_imgs=16, multi_scale=False, name='yolov5s_results', noautoanchor=False, nosave=False,

#### Zip

In [54]:
from datetime import datetime
now = datetime.now() # current date and time
timestamp = now.strftime(" %y-%m-%d %H-%M-%S")
zipname = repo.split("/")[-1] + timestamp
zipname

'Roboflow Export (841) 21-03-14 04-47-42'

In [77]:
mkdir(zipname)
for subdir in ["valid", "train"]:
  !mv "./{subdir}" "{zipname}/"

results_folder = join("runs/train", os.listdir("runs/train")[-1])
!mv "./{results_folder}" "{zipname}/"

!zip "{zipname}.zip" "{zipname}"
cp "{zipname}.zip" "/content/drive/MyDrive/Coding/01_train"

mv: cannot stat './valid': No such file or directory
mv: cannot stat './train': No such file or directory


IndexError: ignored